In [1]:
""" is this script even needed? maybe there are weights already in the files """

import icecube
from icecube import icetray, dataio
from icecube import MuonGun
from icecube.icetray import I3Tray
import glob
import os

<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<char, std::allocator<char> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> >, std::allocator<std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > > > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<OMKey, std::allocator<OMKey> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<short, std::allocator<short> > already registered; second conversion method ignored.
<frozen importlib._bootstrap>:241: RuntimeWarning: to-Python converter for std::vector<int, std::allocator<int> > already registered; second conversion method ignore

In [2]:
def harvest_generators(infiles):
    """
    Harvest serialized generator configurations from a set of I3 files.
    """
    from icecube.icetray.i3logging import log_info as log
    generator = None
    for fname in infiles:
        f = dataio.I3File(fname)
        fr = f.pop_frame(icetray.I3Frame.Stream('S'))
        f.close()
        if fr is not None:
            for k in fr.keys():
                v = fr[k]
                if isinstance(v, MuonGun.GenerationProbability):
                    log('%s: found "%s" (%s)' % (fname, k, type(v).__name__), unit="MuonGun")
                    if generator is None:
                        generator = v
                    else:
                        generator += v
    return generator

In [3]:
model = MuonGun.load_model('Hoerandel5_atmod12_SIBYLL')

datadir = "/data/user/axelpo/LLP-data/"
folderlist = [
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e2_1e3_231129.200717492",
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e2_1e3_231129.200717487",
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e3_1e6_231129.200717474",
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e3_1e6_231129.200717483",
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e4_1e7_gap_100_231129.200719028",
             ]

# dataset enum
#for foldername in folderlist:
foldername = folderlist[0]
datapath = datadir + foldername + "/"
infiles = list(glob.glob(datapath+"LLPSim*/*.gz"))
generator = harvest_generators(infiles)
print(generator)

In [ ]:
model = MuonGun.load_model('Hoerandel5_atmod12_SIBYLL')

datadir = "/data/user/axelpo/LLP-data/"
folderlist = [
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e2_1e3_231129.200717492",
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e2_1e3_231129.200717487",
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e3_1e6_231129.200717474",
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e3_1e6_231129.200717483",
            "DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e4_1e7_gap_100_231129.200719028",
             ]

# dataset enum
for foldername in folderlist:
    datapath = datadir + foldername + "/"
    infiles = list(glob.glob(datapath+"LLPSim*/*.gz"))
    #print(infiles)
    generator = harvest_generators(infiles)
    
    print(generator)
    
    print("Doing folder", foldername)
    tray = I3Tray()
    tray.Add("I3Reader", filenamelist=infiles)
    tray.AddModule('I3MuonGun::WeightCalculatorModule', 'MuonWeight', Model=model,
        Generator=generator)
    tray.Add("I3Writer", filename=datapath+"Trigger_weighted.i3.gz")
    tray.Execute()

In [ ]:
"""
#infiles = list(glob.glob(datapath+"LLPSim*/*.gz"))
model = MuonGun.load_model('Hoerandel5_atmod12_SIBYLL')
infiles = ["/data/user/axelpo/LLP-data/DarkLeptonicScalar.mass-130.eps-5e-6.nevents-10000_ene_1e2_1e3_231129.200717492/L2.i3.gz"]

generator = harvest_generators(infiles)

tray = I3Tray()
tray.Add("I3Reader", filenamelist=infiles)
tray.AddModule('I3MuonGun::WeightCalculatorModule', 'MuonWeight', Model=model,
    Generator=generator)
tray.Add("I3Writer", filename="L2_weighted.i3.gz")
tray.Execute()
"""